In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
df=pd.read_csv('news.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [25]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [26]:
df=df.dropna()

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18285 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18285 non-null  int64 
 1   title   18285 non-null  object
 2   author  18285 non-null  object
 3   text    18285 non-null  object
 4   label   18285 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 857.1+ KB


In [28]:
X=df.drop(['label'],axis=1)

In [29]:
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [30]:
y=df['label']

In [31]:
X.shape,y.shape

((18285, 4), (18285,))

In [34]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [35]:
voc_size=5000

## One hot representation

In [47]:
message=X.copy()

In [48]:
message['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [49]:
message.reset_index(inplace=True)

In [50]:
import nltk 
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [52]:
message=message.drop(['index','id'],axis=1)
message.head()

,title,author,text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [54]:
wordnet=WordNetLemmatizer()
corpus=[]

for i in range(0,len(message)):
    review=re.sub('[^a-zA-Z]',' ',message['title'][i])
    review=review.lower()
    review=review.split()
    review=[wordnet.lemmatize(word)for word in review if  not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)


In [55]:
corpus

['house dem aide even see comey letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 'civilian killed single u airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathroom exclusive video breitbart',
 'beno hamon win french socialist party presidential nomination new york time',
 'back channel plan ukraine russia courtesy trump associate new york time',
 'obama organizing action partner soros linked indivisible disrupt trump agenda',
 'bbc comedy sketch real housewife isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter arctic photo',
 'u official see link trump russia',
 'yes paid government troll social medium blog forum website',
 'major league soccer argentine find home success new york time',
 'well fargo chief abruptly step new york time',
 'anonymous donor pay 

In [56]:
voc_size=5000
onehot=[one_hot(word,voc_size)for word in corpus]
onehot

[[3305, 1668, 2122, 556, 2851, 790, 4149, 2693, 863, 3628],
 [3574, 2068, 2047, 1844, 3177, 600, 4039],
 [3297, 2662, 461, 3921],
 [3612, 224, 1111, 2268, 3863, 1005],
 [1422, 3177, 18, 307, 2794, 204, 3177, 3588, 3285, 1742],
 [4435,
  1620,
  4951,
  869,
  4065,
  711,
  4817,
  2373,
  3276,
  1867,
  4618,
  4383,
  4092,
  2886,
  4039],
 [1772, 3057, 4872, 3949, 797, 4522, 3119, 2327, 22, 1666, 3743],
 [3479, 3697, 1795, 3008, 1591, 4178, 711, 702, 22, 1666, 3743],
 [1066, 124, 1353, 2407, 2397, 4913, 1820, 896, 711, 706],
 [1944, 34, 2632, 1381, 2143, 4269, 3193, 1168],
 [3938, 3765, 1600, 1432, 542, 4768, 3749, 3450, 1524, 2289, 3752],
 [2268, 494, 2851, 3491, 711, 1591],
 [4355, 4467, 4441, 4077, 2204, 3988, 3091, 3190, 3744],
 [1147, 3819, 948, 719, 3909, 4339, 2578, 22, 1666, 3743],
 [2977, 1539, 3501, 2933, 3490, 22, 1666, 3743],
 [3503, 1359, 966, 1527, 646, 1167, 4738, 3601, 4687, 4704],
 [1805, 1392, 2068],
 [3127, 1376, 2131, 3087, 711, 3089, 1690, 4039],
 [2600, 408, 

## Word Embedding Representation

In [57]:
sent_length=20
embedded_docs=pad_sequences(onehot,padding='pre',maxlen=sent_length)
print(embedded_docs[0])

[   0    0    0    0    0    0    0    0    0    0 3305 1668 2122  556
 2851  790 4149 2693  863 3628]


### creating lstm model

In [58]:
dimension=40
model=Sequential()
model.add(Embedding(voc_size,dimension,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [59]:
len(embedded_docs),y.shape

(18285, (18285,))

In [60]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [65]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [66]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 20s 81ms/step - loss: 0.3427 - accuracy: 0.8337 - val_loss: 0.1955 - val_accuracy: 0.9162
Epoch 2/10
192/192 [==============================] - 12s 63ms/step - loss: 0.1325 - accuracy: 0.9469 - val_loss: 0.2045 - val_accuracy: 0.9229
Epoch 3/10
192/192 [==============================] - 10s 53ms/step - loss: 0.0874 - accuracy: 0.9681 - val_loss: 0.2297 - val_accuracy: 0.9130
Epoch 4/10
192/192 [==============================] - 13s 69ms/step - loss: 0.0563 - accuracy: 0.9814 - val_loss: 0.2961 - val_accuracy: 0.9117
Epoch 5/10
192/192 [==============================] - 18s 96ms/step - loss: 0.0417 - accuracy: 0.9866 - val_loss: 0.2712 - val_accuracy: 0.9181
Epoch 6/10
192/192 [==============================] - 16s 81ms/step - loss: 0.0268 - accuracy: 0.9924 - val_loss: 0.3955 - val_accuracy: 0.9094
Epoch 7/10
192/192 [==============================] - 11s 55ms/step - loss: 0.0142 - accuracy: 0.9957 - val_loss: 0.4838 - val_accuracy:

In [82]:
y_pred=model.predict(X_test)
y_pred

189/189 [==============================] - 5s 24ms/step


array([[9.9995118e-01],
       [1.9148592e-05],
       [4.8721940e-03],
       ...,
       [7.6779515e-06],
       [9.9988747e-01],
       [9.9999946e-01]], dtype=float32)

In [83]:
from sklearn.metrics import confusion_matrix,classification_report

confusion_matrix(y_test,y_pred)

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [84]:
print(classification_report(y_test,y_pred))

ValueError: Classification metrics can't handle a mix of binary and continuous targets